In [7]:
import torch
model = torch.hub.load('pytorch/vision:v0.10.0', 'alexnet', pretrained=False)
model.eval()

Using cache found in /Users/kilometers/.cache/torch/hub/pytorch_vision_v0.10.0


AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [8]:
import urllib
url, filename = ("https://github.com/pytorch/hub/raw/master/images/dog.jpg", "dog.jpg")
try: urllib.URLopener().retrieve(url, filename)
except: urllib.request.urlretrieve(url, filename)

In [9]:
# sample execution (requires torchvision)
from PIL import Image
from torchvision import transforms
input_image = Image.open(filename)
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
input_tensor = preprocess(input_image)
input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model

# move the input and model to GPU for speed if available
if torch.cuda.is_available():
    input_batch = input_batch.to('cuda')
    model.to('cuda')

with torch.no_grad():
    output = model(input_batch)
# Tensor of shape 1000, with confidence scores over Imagenet's 1000 classes
print(output[0])
# The output has unnormalized scores. To get probabilities, you can run a softmax on it.
probabilities = torch.nn.functional.softmax(output[0], dim=0)
print(probabilities)

tensor([ 9.7420e-03, -1.1239e-03, -4.8311e-03, -4.6106e-03, -5.4522e-03,
        -4.0637e-03,  3.9791e-03,  8.7456e-03, -1.7287e-02,  8.0074e-03,
         2.9100e-04, -1.2014e-02,  5.0973e-03, -9.5425e-03, -6.6392e-05,
        -1.9646e-03,  8.6980e-03, -2.6864e-05,  2.4202e-03,  4.9970e-04,
         5.4614e-03,  1.0872e-02, -5.1152e-03,  1.9442e-03,  1.2683e-02,
        -4.7835e-03,  1.3186e-02, -1.3646e-02,  1.7957e-03, -1.2067e-02,
         1.2467e-02, -5.5136e-03,  1.7632e-02,  1.4596e-02,  1.3558e-02,
         1.7546e-03, -1.4020e-03, -2.2713e-02, -1.9184e-02, -1.4776e-03,
        -4.4833e-03,  7.2052e-03,  1.5196e-02, -7.4724e-03,  9.1399e-03,
         2.9571e-03, -9.2198e-03,  1.8065e-02, -3.1992e-03, -8.6710e-03,
         7.2788e-03,  1.3491e-02,  1.4511e-02,  1.1313e-02, -1.9356e-02,
         7.3327e-03,  2.6454e-03,  1.7585e-02,  3.6348e-03, -1.0446e-02,
        -8.8192e-03, -1.6475e-02,  1.5117e-02, -2.6343e-03, -4.1062e-03,
        -8.2393e-03,  9.0381e-03,  1.7972e-02, -2.6

In [11]:
!wget https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt


zsh:1: command not found: wget


In [12]:
# Read the categories
with open("imagenet_classes.txt", "r") as f:
    categories = [s.strip() for s in f.readlines()]
# Show top categories per image
top5_prob, top5_catid = torch.topk(probabilities, 5)
for i in range(top5_prob.size(0)):
    print(categories[top5_catid[i]], top5_prob[i].item())

FileNotFoundError: [Errno 2] No such file or directory: 'imagenet_classes.txt'